Давыдов ПИм-231 лаба 17

Ансамблевые методы: Stacking(стекинг). Для выполнения задания возьмите датасет о заболевании сахарным диабетом. В качестве "слабых" учеников используйте ранее изученные методы для классификации .

(1) Выполните масштабирование данных.

(2) 1-ю конфигурацию сделайте следующую: в качестве слабых учеников возьмите SVM, KNN, метод решающих деревьев с параметрами по умолчанию, в качестве метамодели - логистическую регрессию.

(3) 2-ю конфигурацию возьмите с таким же набором слабых учеников, но для каждого задайте наилучшие гиперпараметры на основе выполненных ранее заданий.

(4) 3-ю конфигурацию - возьмите по два слабых ученика по каждой модели (одна с гиперпараметрами по умолчанию, вторая - с наилучшими).

(5) Для каждого ансамбля моделей выведите его конфигурацию.

(6) Проведите сравнение (classification_report) с ранее выполненными работами. Позволило ли существенно улучшить качество прогноза заболевания такое усовершенствование алгоритма? Если есть возможность выполнить подбор гиперпараметров модели - сделайте!

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression


(1) Выполните масштабирование данных.

In [ ]:
data = pd.read_csv("diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

standard = StandardScaler()
standard.fit(X)
standard_x = standard.transform(X)

X_train, X_test, y_train, y_test = train_test_split(standard_x, y, test_size=0.3, random_state=42)

(2) 1-ю конфигурацию сделайте следующую: в качестве слабых учеников возьмите SVM, KNN, метод решающих деревьев с параметрами по умолчанию, в качестве метамодели - логистическую регрессию.

In [ ]:
estimators = [
    ('svm', SVC()),  # SVM
    ('knn', KNeighborsClassifier()),  # KNN
    ('decision_tree', DecisionTreeClassifier())  # Решающее дерево
]

meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=estimators, final_estimator=meta_model)
stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)
print("Конфигурация ансамбля: ", stacking_model.estimators_, " ---> ", stacking_model.final_estimator_)

              precision    recall  f1-score   support

           0       0.80      0.81      0.81       151
           1       0.63      0.62      0.63        80

    accuracy                           0.74       231
   macro avg       0.72      0.72      0.72       231
weighted avg       0.74      0.74      0.74       231

Конфигурация ансамбля:  [SVC(), KNeighborsClassifier(), DecisionTreeClassifier()]  --->  LogisticRegression()


(3) 2-ю конфигурацию возьмите с таким же набором слабых учеников, но для каждого задайте наилучшие гиперпараметры на основе выполненных ранее заданий.

In [ ]:
estimators = [
    ('svm', SVC(kernel='linear', C = 1)),  # SVM
    ('knn', KNeighborsClassifier(n_neighbors=29)),  # KNN
    ('decision_tree', DecisionTreeClassifier(max_depth=4))  # Решающее дерево
]
# подставили лучшие параметры
meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=estimators, final_estimator=meta_model)
stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)
print("Конфигурация ансамбля: ", stacking_model.estimators_, " ---> ", stacking_model.final_estimator_)

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       151
           1       0.65      0.64      0.64        80

    accuracy                           0.75       231
   macro avg       0.73      0.73      0.73       231
weighted avg       0.75      0.75      0.75       231

Конфигурация ансамбля:  [SVC(C=1, kernel='linear'), KNeighborsClassifier(n_neighbors=29), DecisionTreeClassifier(max_depth=4)]  --->  LogisticRegression()


(4) 3-ю конфигурацию - возьмите по два слабых ученика по каждой модели (одна с гиперпараметрами по умолчанию, вторая - с наилучшими).

In [ ]:
estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('decision_tree', DecisionTreeClassifier()),
    ('svm_best', SVC(kernel='linear', C = 1)),
    ('knn_best', KNeighborsClassifier(n_neighbors=29)),
    ('decision_tree_best', DecisionTreeClassifier(max_depth=4))
]

meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=estimators, final_estimator=meta_model)
stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)
print("Конфигурация ансамбля: ", stacking_model.estimators_, " ---> ", stacking_model.final_estimator_)

              precision    recall  f1-score   support

           0       0.80      0.81      0.81       151
           1       0.64      0.61      0.62        80

    accuracy                           0.74       231
   macro avg       0.72      0.71      0.72       231
weighted avg       0.74      0.74      0.74       231

Конфигурация ансамбля:  [SVC(), KNeighborsClassifier(), DecisionTreeClassifier(), SVC(C=1, kernel='linear'), KNeighborsClassifier(n_neighbors=29), DecisionTreeClassifier(max_depth=4)]  --->  LogisticRegression()



(5) Для каждого ансамбля моделей выведите его конфигурацию.

Сделал.

(6) Проведите сравнение (classification_report) с ранее выполненными работами. Позволило ли существенно улучшить качество прогноза заболевания такое усовершенствование алгоритма? Если есть возможность выполнить подбор гиперпараметров модели - сделайте!

Существенных улучшений нет.

In [ ]:
param_grid = {'cv': [3, 4, 5, 6, 7]}

grid_search = GridSearchCV(stacking_model, param_grid, scoring='accuracy', cv=5)

grid_search.fit(X_train, y_train)

# Вывод наилучшего значения параметра cv
print("Лучшее значение параметра cv:", grid_search.best_params_['cv'])

Лучшее значение параметра cv: 4


In [ ]:
estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('decision_tree', DecisionTreeClassifier()),
    ('svm_best', SVC(kernel='linear', C = 1)),
    ('knn_best', KNeighborsClassifier(n_neighbors=29)),
    ('decision_tree_best', DecisionTreeClassifier(max_depth=4))
]

meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=estimators, final_estimator=meta_model, cv=4)
stacking_model.fit(X_train, y_train)

y_pred = stacking_model.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)
print("Конфигурация ансамбля: ", stacking_model.estimators_, " ---> ", stacking_model.final_estimator_)

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       151
           1       0.64      0.64      0.64        80

    accuracy                           0.75       231
   macro avg       0.72      0.72      0.72       231
weighted avg       0.75      0.75      0.75       231

Конфигурация ансамбля:  [SVC(), KNeighborsClassifier(), DecisionTreeClassifier(), SVC(C=1, kernel='linear'), KNeighborsClassifier(n_neighbors=29), DecisionTreeClassifier(max_depth=4)]  --->  LogisticRegression()
